In [2]:
!pip install mpl_toolkits

  Could not find a version that satisfies the requirement mpl_toolkits (from versions: )
No matching distribution found for mpl_toolkits


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.basemap import Basemap

import folium
from folium import plugins
from folium import features
from geojson import Polygon
from folium import IFrame
import base64

import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import interactive


ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [2]:
# Данные о реальном количестве поездок
data_real = pd.read_csv('2016-06 102reg.csv', index_col=0, parse_dates=True)
data_real.head(3)

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-06-01 00:00:00,26,30,19,39,71,163,181,219,326,56,...,1,1,6,262,107,114,7,169,6,85
2016-06-01 01:00:00,14,21,6,26,49,101,136,144,252,27,...,5,0,0,137,1,27,16,105,4,42
2016-06-01 02:00:00,5,25,4,18,25,42,74,123,245,30,...,8,0,1,25,0,11,0,31,0,10


In [3]:
# Преобразование данных для дальнейшего использования
df_real = data_real.copy()
df_real = df_real.reset_index()
df_real.rename(columns={'index':'DateTime'},inplace=True)
df_real = pd.melt(df_real,id_vars='DateTime',var_name="zone",value_name="Trips")

df_real.zone = df_real.zone.astype(int)

df_real['Day'] = df_real.DateTime.dt.day
df_real['Hour'] = df_real.DateTime.dt.hour

df_real.head(3)


,DateTime,zone,Trips,Day,Hour
0,2016-06-01 00:00:00,1075,26,1,0
1,2016-06-01 01:00:00,1075,14,1,1
2,2016-06-01 02:00:00,1075,5,1,2


In [4]:
# Результаты моделирования с 6й недели
data_pred = pd.read_csv('Predictions.csv')
data_pred.head(3)

,zone,Day,Hour,shift_h,Predict
0,1075,31,23,1,30.500000
1,1075,31,23,2,20.171429
2,1075,31,23,3,13.828571


In [5]:
# Таблица 102 зон
regions = pd.read_csv('region_coord.csv')
regions.head(3)

,zone,west,east,south,north
0,1075,-74.022246,-74.011135,40.697437,40.705825
1,1076,-74.022246,-74.011135,40.705825,40.714213
2,1077,-74.022246,-74.011135,40.714213,40.722601


In [6]:
# Функция для зпаиси координат в виде, необходимом для формирования полигона - формат WKT, 
#            well known text (подробнее — https://en.wikipedia.org/wiki/Well-known_text).

def poly_line(line):
    return [[[line[1],line[3]], [line[1],line[4]], [line[2],line[4]], [line[2],line[3]], [line[1],line[3]]]]

In [7]:
# добавим столбец с геометрией
regions['geometry'] = regions.apply(poly_line,axis=1)
regions.head()
#regions.index = regions.index.astype(str)

,zone,west,east,south,north,geometry
0,1075,-74.022246,-74.011135,40.697437,40.705825,"[[[-74.0222464, 40.6974368], [-74.0222464, 40...."
1,1076,-74.022246,-74.011135,40.705825,40.714213,"[[[-74.0222464, 40.705825], [-74.0222464, 40.7..."
2,1077,-74.022246,-74.011135,40.714213,40.722601,"[[[-74.0222464, 40.7142132], [-74.0222464, 40...."
3,1125,-74.011135,-74.000023,40.697437,40.705825,"[[[-74.0111348, 40.6974368], [-74.0111348, 40...."
4,1126,-74.011135,-74.000023,40.705825,40.714213,"[[[-74.0111348, 40.705825], [-74.0111348, 40.7..."


In [8]:
# Функция преобразования в формат json

def df_to_geojson(df, properties, geom='geometry'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Polygon',
                               'coordinates':[]}}
        feature['geometry'] = (Polygon(row[geom]))
        for prop in properties:
            feature['properties'][prop] = row[prop]
       
        geojson['features'].append(feature)
    return geojson

In [9]:
# Рисуем карту с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени
def map_r(h=12, d=1, sh=1):
    
    df_map = pd.merge(df_real[(df_real.Day==d) & (df_real.Hour==h)], regions[['zone', 'geometry']] , on='zone')
    
    df_pred = data_pred[data_pred.shift_h==sh]
    df_pred = pd.merge(df_pred[(df_pred.Day.shift(-sh)==d) & (df_pred.Hour.shift(-sh)==h)], 
                       regions[['zone', 'geometry']] , on='zone')
        
    m = folium.Map(location=[40.74, -73.89],zoom_start=12, width=900, height=800, tiles='Stamen Toner')
              
    m.choropleth(
        geo_data=df_to_geojson(df_map, ['zone']),
        name='choropleth',
        data=df_map[['zone', 'Trips']],
        #threshold_scale= [0, 10, 50, 100, 200, 500],
        key_on='feature.properties.zone',
        columns=['zone', 'Trips'],
        fill_color='RdGy_r',
        line_weight=0.5,
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name='Trips',
        highlight = True
    )
    
    for i in range(0, 102):
    
        gj = folium.GeoJson(
            data={
                'type': 'Polygon',
                'coordinates': df_map.geometry.iloc[i]
            }
        )
        
        popup = (
            "Zone: {z}<br>"
            "Time Shift: {shft}<br>"
            "Real Trips: {rt}<br>"
            "Prediction: {pr:.2f}<br>"
            "Residual: {res:.2f} %<br>"
               ).format(z=df_map.zone[i],
                shft=sh,        
                rt=df_map.Trips[i],
                pr=df_pred.Predict[i],
                res=(100*(df_map.Trips[i] - df_pred.Predict[i])/df_map.Trips[i]),
            )
        
        gj.add_child(folium.Popup(popup))
        gj.add_to(m)

    m.save('trips.html')
    return m

In [10]:
# Рисуем карту для визуализации временного ряд фактического и прогнозируемого спроса на такси в выбираемой области
def map_graph( sh=1):
    
    resolution, width, height = 75, 8, 5
    
    df_avg = pd.DataFrame(data_real.mean(), columns=['AvgTrips'])
    df_avg['zone'] = df_avg.index.astype(int)
     
    df_map = pd.merge(df_avg, regions[['zone', 'geometry']] , on='zone')
        
    m = folium.Map(location=[40.74, -73.89],zoom_start=12, width=900, height=800, tiles='Stamen Toner')
              
    m.choropleth(
        geo_data=df_to_geojson(df_map, ['zone']),
        name='choropleth',
        data=df_map[['zone', 'AvgTrips']],
        key_on='feature.properties.zone',
        columns=['zone', 'AvgTrips'],
        fill_color='RdGy_r',
        line_weight=0.5,
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name='Average Trip Quantity',
        highlight = True
    )
    
    for z in regions.zone:

        gj = folium.GeoJson(
            data={
                'type': 'Polygon',
                'coordinates': df_map[df_map['zone']==z].geometry.iloc[0]
            }
        )
       
        df_pred1 = data_pred[data_pred.shift_h==sh]
        df_real1 = df_real.copy()

        index = df_real1.DateTime[df_real1.zone==z]

        plot_real = pd.DataFrame(index=index, data=df_real1[df_real1.zone==z].shift(-(sh-1)).Trips.values[:len(index)])
        plot_pred = pd.DataFrame(index=index, data=df_pred1[df_pred1.zone==z].Predict.values[:len(index)])
        
        fig, ax = plt.subplots(figsize=(width, height))
        ax = plot_real.plot(ax=ax)
        ax = plot_pred.plot(ax=ax)
        ax.legend(['Real','Prediction'])
        ax.set_title('Zone: '+str(z)+', Shift on '+str(sh)+'hours')
        ax.set_ylabel('Trips')

        png = 'mpld3_{}.png'.format('42')
        fig.savefig(png, dpi=75)
        plt.close(fig)
    
        encoded = base64.b64encode(open(png, 'rb').read())
        html = '<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
            
        gj.add_child(folium.Popup(iframe, max_width=2650))
        gj.add_to(m)
#
    
    m.save('graph.html')
    return m

In [11]:
# Интерактивная карта с ползунками для сдвига, дня и часа
d_slider = widgets.IntSlider(min=1, max=30, step=1, value=1)

h_slider = widgets.Dropdown(
    options=range(0,24),
    value=12,
    description='Hours:',
    disabled=False, orientation='vertical'
)

d_slider = widgets.Dropdown(
    options=range(1,31),
    value=1,
    description='Day of month:',
    disabled=False, orientation='vertical'
)

shift_slider = widgets.Dropdown(
    options=range(1,7),
    value=1,
    description='Shift in Hours:',
    disabled=False
)

widgets.interact(map_r, h=h_slider, d=d_slider, sh=shift_slider)

interactive(children=(Dropdown(description=u'Hours:', index=12, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23), value=12), Dropdown(description=u'Day of month:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30), value=1), Dropdown(description=u'Shift in Hours:', options=(1, 2, 3, 4, 5, 6), value=1), Output()), _dom_classes=('widget-interact',))

<function __main__.map_r>

In [ ]:
# Интерактивная карта с ползуном для сдвига
shift_slider_g = widgets.Dropdown(
    options=range(1,7),
    value=1,
    description='Shift in Hours:',
    disabled=False
)

widgets.interact(map_graph, sh=shift_slider_g)